In [2]:
# Check the gene names
import pandas as pd
import requests

# Read Document 1 (assumes an Excel file with a column "Protein" containing gene names)
df1 = pd.read_excel('/Users/conny/Desktop/AlphaFold/Summer Project/data/Paper 2 data.xlsx', header=11)

# Now you can access the 'Protein' column
protein_columns = df1['T: Majority protein IDs'].dropna().unique()

#print(protein_columns)
#Get the first entry inside each list
# Extract the first protein ID from each entry
protein_ids = [entry.split(';')[0].strip() for entry in protein_columns]

# Now protein_ids will contain only the first ID from each entry
print(protein_ids)
#gene_names = df1['Protein'].dropna().unique()  # unique gene names, drop NaNs if any


['Q8TDN6', 'Q6PK04', 'P46087', 'Q92522', 'Q9NVP1', 'O76021', 'Q9NQ55', 'Q9NR30', 'O00567', 'Q9BYG3', 'Q9Y5J1', 'Q9UKD2', 'Q13838', 'Q9BQG0', 'Q15050', 'P22087', 'Q9UKM9', 'Q9H7B2', 'P38159', 'O43390', 'Q5T3I0', 'Q9BXY0', 'Q9Y3B9', 'Q8IY81', 'P38919', 'Q08211', 'Q8WWQ0', 'P55769', 'O95478', 'Q9NX58', 'P42696', 'Q9GZL7', 'Q9HCE1', 'Q99729-3', 'Q8WYP5', 'Q13310-2', 'Q9Y3C1', 'Q8TDD1-2', 'Q9NY12', 'P07910-2', 'Q9BRT6', 'Q15061', 'P62314', 'O00571', 'Q9Y3A4', 'P17844', 'Q15717', 'Q9H9Y2', 'Q13823', 'Q9BWF3', 'Q14684', 'Q14137', 'P16401', 'Q13601', 'O75683', 'P19525', 'P31942', 'O15226', 'G5E9M5', 'Q96KR1', 'Q9NUL3', 'Q86V81', 'Q9H6F5', 'Q9NY93', 'P14678', 'P52272', 'P50990', 'O75367', 'Q14692', 'O60506', 'Q8TED0', 'Q9BVJ6', 'Q15029', 'Q13610', 'Q09161', 'Q12905', 'P61978', 'Q1KMD3', 'P61326', 'P84090', 'Q9Y2P8', 'O76094', 'O96028', 'O14979', 'Q9H8H2', 'O15213', 'Q14739', 'Q14690', 'O43143', 'Q9NVX2', 'Q5BKZ1', 'Q14151', 'P49711', 'Q92900', 'P51114', 'P07910', 'Q9NV31', 'P62316', 'Q9Y5S9', '

In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import sys
import json
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import random

def fetch_sequence(protein_id):
    """
    Fetch sequence from UniProt
    """
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    params = {"fields": ["sequence"]}
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        data = response.json()
        sequence_info = data.get("sequence", {})
        return sequence_info.get("value", None)
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None

def main():
    # Read the Excel file
    excel_file = "/Users/conny/Desktop/AlphaFold/Summer Project/data/Paper 2 data.xlsx"
    
    # Read the relevant columns
    df = pd.read_excel(excel_file, header=11)
    
    # Extract protein IDs and descriptions
    protein_data = []
    for _, row in df.iterrows():
        protein_ids = row['T: Majority protein IDs'].split(';')
        description = row['T: Protein names']
        
        # Get the first protein ID from each entry
        primary_id = protein_ids[0].strip()
        protein_data.append({
            'accession': primary_id,
            'description': description
        })
    
    # Get sequences and create results
    results = []
    for entry in tqdm(protein_data, desc="Fetching sequences"):
        sequence = fetch_sequence(entry['accession'])
        if sequence:
            results.append({
                'accession': entry['accession'],
                'description': entry['description'],
                'sequence': sequence
            })
    
    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("paper2_reviewed_results.csv", index=False)
    
    # Save sequences to FASTA file
    with open("paper2_protein_sequences.fasta", "w") as f:
        for result in results:
            f.write(f">{result['accession']}\n")
            f.write(f"{result['sequence']}\n")
    
    print("\nResults summary:")
    print(f"- Total proteins processed: {len(protein_data)}")
    print(f"- Successfully retrieved: {len(results)} sequences")
    print("\nOutput files:")
    print("- paper2_reviewed_results.csv: Contains all retrieved entries with descriptions")
    print("- paper2_protein_sequences.fasta: Contains protein sequences in FASTA format")

if __name__ == "__main__":
    main()

Fetching sequences: 100%|██████████| 469/469 [03:12<00:00,  2.43it/s]


Results summary:
- Total proteins processed: 469
- Successfully retrieved: 466 sequences

Output files:
- paper2_reviewed_results.csv: Contains all retrieved entries with descriptions
- paper2_protein_sequences.fasta: Contains protein sequences in FASTA format


In [5]:
import pandas as pd

# Read the original Paper 2 data
paper2_data = pd.read_excel('/Users/conny/Desktop/AlphaFold/Summer Project/data/Paper 2 data.xlsx', header=11)

# Read the retrieved sequences
retrieved_sequences = pd.read_csv('paper2_reviewed_results.csv')

# Get unique proteins from both datasets
original_proteins = protein_ids
retrieved_proteins = list(set(retrieved_sequences['accession'].unique()))

# Find proteins that weren't retrieved
missing_proteins = list(set(original_proteins) - set(retrieved_proteins))  # This works

# Create a DataFrame with the missing proteins and their descriptions
#missing_df = paper2_data[paper2_data['gene'].isin(missing_proteins)][['gene', 'description']]

# Print summary
print(f"Total proteins in original data: {len(original_proteins)}")
print(f"Successfully retrieved proteins: {len(retrieved_proteins)}")
print(f"Missing proteins: {(missing_proteins)}")
print("\nMissing proteins:")
#print(missing_df.to_string())

# Save missing proteins to CSV for further analysis
#missing_df.to_csv("paper2_missing_proteins.csv", index=False)
#print("\nSaved missing proteins to paper2_missing_proteins.csv")

Total proteins in original data: 469
Successfully retrieved proteins: 466
Missing proteins: ['Q5HYZ1', 'P08107', 'Q5T6W5']

Missing proteins:


In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import sys
import json
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import random

def fetch_sequence_and_info(protein_id):
    """
    Fetch sequence and protein information from UniProt
    """
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    params = {"fields": ["sequence", "gene_names", "protein_name"]}
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        data = response.json()
        print(data)
        sequence = data.get("sequence", {}).get("value", None)
        gene_names = data.get("genes", [{}])[0].get("geneName", {}).get("value", "")
        protein_name = data.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value", "")
        return sequence, gene_names, protein_name
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None, None, None

def main():
    # Read the Excel file
    excel_file = "/Users/conny/Desktop/AlphaFold/Summer Project/data/Paper 2 data.xlsx"
    
    # Read the relevant columns
    df = pd.read_excel(excel_file, header=11)
    
    # Extract protein IDs and descriptions
    protein_data = []
    for _, row in df.iterrows():
        protein_ids = row['T: Majority protein IDs'].split(';')
        description = row['T: Protein names']
        
        # Get the first protein ID from each entry
        primary_id = protein_ids[0].strip()
        protein_data.append({
            'accession': primary_id,
            'description': description
        })
    
    # Get sequences and create results
    results = []
    for entry in tqdm(protein_data, desc="Fetching sequences"):
        sequence, gene_name, protein_name = fetch_sequence_and_info(entry['accession'])
        if sequence:
            results.append({
                'accession': entry['accession'],
                'gene': gene_name,  # Added gene name
                'description': entry['description'],
                'protein_name': protein_name,  # Added protein name
                'sequence': sequence
            })
    
    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("paper2_fasta_results.csv", index=False)
    
    # Save sequences to FASTA file
    with open("paper2_fasta_results.fasta", "w") as f:
        for result in results:
            f.write(f">{result['accession']} {result['gene']} {result['protein_name']}\n")
            f.write(f"{result['sequence']}\n")
    
    print("\nResults summary:")
    print(f"- Total proteins processed: {len(protein_data)}")
    print(f"- Successfully retrieved: {len(results)} sequences")
    print("\nOutput files:")
    print("- paper2_fasta_results.csv: Contains all retrieved entries with gene names, accessions, descriptions, and sequences")
    print("- paper2_fasta_results.fasta: Contains protein sequences in FASTA format with gene names in headers")

if __name__ == "__main__":
    main()

Fetching sequences:   0%|          | 1/469 [00:00<03:24,  2.29it/s]

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'Q8TDN6', 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Ribosome biogenesis protein BRX1 homolog'}}, 'alternativeNames': [{'fullName': {'value': 'Brix domain-containing protein 2'}}]}, 'genes': [{'geneName': {'value': 'BRIX1'}, 'synonyms': [{'value': 'BRIX'}, {'value': 'BXDC2'}]}], 'sequence': {'value': 'MAATKRKRRGGFAVQAKKPKRNEIDAEPPAKRHATAEEVEEEERDRIPGPVCKGKWKNKERILIFSSRGINFRTRHLMQDLRMLMPHSKADTKMDRKDKLFVINEVCEMKNCNKCIYFEAKKKQDLYMWLSNSPHGPSAKFLVQNIHTLAELKMTGNCLKGSRPLLSFDPAFDELPHYALLKELLIQIFSTPRYHPKSQPFVDHVFTFTILDNRIWFRNFQIIEEDAALVEIGPRFVLNLIKIFQGSFGGPTLYENPHYQSPNMHRRVIRSITAAKYREKQQVKDVQKLRKKEPKTLLPHDPTADVFVTPAEEKPIEIQWVKPEPKVDLKARKKRIYKRQRKMKQRMDSGKTK', 'length': 353, 'molWeight': 41401, 'crc64': 'F05597673A13B7CC', 'md5': '75B4A1FC32ED97A1B0C124B39BC4CBEE'}, 'extraAttributes': {'uniParcId': 'UPI000007186B'}}


Fetching sequences:   0%|          | 2/469 [00:00<03:23,  2.29it/s]

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'Q6PK04', 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Coiled-coil domain-containing protein 137'}}}, 'genes': [{'geneName': {'value': 'CCDC137'}, 'synonyms': [{'evidences': [{'evidenceCode': 'ECO:0000303', 'source': 'PubMed', 'id': '20813266'}], 'value': 'cPERP-B'}]}], 'sequence': {'value': 'MAGAGRGAAVSRVQAGPGSPRRARGRQQVQPLGKQRPAPWPGLRSKEKKKVNCKPKNQDEQEIPFRLREIMRSRQEMKNPISNKKRKKAAQVTFRKTLEKEAKGEEPDIAVPKFKQRKGESDGAYIHRMQQEAQHVLFLSKNQAIRQPEVQAAPKEKSEQKKAKKAFQKRRLDKVRRKKEEKAADRLEQELLRDTVKFGEVVLQPPELTARPQRSVSKDQPGRRSQMLRMLLSPGGVSQPLTASLARQRIVEEERERAVQAYRALKQRQQQLHGERPHLTSRKKPEPQL', 'length': 289, 'molWeight': 33231, 'crc64': 'B8C4E6EC64360771', 'md5': '62B4EF81784A75BC0A239598345511D7'}, 'extraAttributes': {'uniParcId': 'UPI00001983C7'}}


Fetching sequences:   1%|          | 3/469 [00:01<03:24,  2.28it/s]

...

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'P62834', 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Ras-related protein Rap-1A'}, 'ecNumbers': [{'evidences': [{'evidenceCode': 'ECO:0000250', 'source': 'UniProtKB', 'id': 'P61224'}], 'value': '3.6.5.2'}]}, 'alternativeNames': [{'fullName': {'value': 'C21KG'}}, {'fullName': {'value': 'G-22K'}}, {'fullName': {'value': 'GTP-binding protein smg p21A'}}, {'fullName': {'value': 'Ras-related protein Krev-1'}}], 'flag': 'Precursor'}, 'genes': [{'geneName': {'value': 'RAP1A'}, 'synonyms': [{'value': 'KREV1'}]}], 'sequence': {'value': 'MREYKLVVLGSGGVGKSALTVQFVQGIFVEKYDPTIEDSYRKQVEVDCQQCMLEILDTAGTEQFTAMRDLYMKNGQGFALVYSITAQSTFNDLQDLREQILRVKDTEDVPMILVGNKCDLEDERVVGKEQGQNLARQWCNCAFLESSAKSKINVNEIFYDLVRQINRKTPVEKKKPKKKSCLLL', 'length': 184, 'molWeight': 20987, 'crc64': '42C39290C98E0A92', 'md5': '3D957362BCF34E22DBB7A44930D36BF4'}, 'extraAttributes': {'uniParcId': 'UPI0000001250'}}


Fetching sequences: 100%|█████████▉| 468/469 [03:34<00:00,  2.23it/s]

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'P26639', 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000305'}], 'value': 'Threonine--tRNA ligase 1, cytoplasmic'}, 'ecNumbers': [{'evidences': [{'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '25824639'}], 'value': '6.1.1.3'}]}, 'alternativeNames': [{'fullName': {'value': 'Threonyl-tRNA synthetase'}, 'shortNames': [{'value': 'ThrRS'}]}, {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000312', 'source': 'HGNC', 'id': 'HGNC:11572'}], 'value': 'Threonyl-tRNA synthetase 1'}}]}, 'genes': [{'geneName': {'evidences': [{'evidenceCode': 'ECO:0000312', 'source': 'HGNC', 'id': 'HGNC:11572'}], 'value': 'TARS1'}, 'synonyms': [{'value': 'TARS'}]}], 'sequence': {'value': 'MFEEKASSPSGKMGGEEKPIGAGEEKQKEGGKKKNKEGSGDGGRAELNPWPEYIYTRLEMYNILKAEHDSILAEKAEKDSKPIKVTLPDGKQVDAESWKTTPYQIACGISQGLADNTVIAKVNNVVWDLDRPLEEDCTLELLKFEDEEAQAVYWHSSAHIMGEAMERVYGGCLCYGPPIENGFYYDMYLEEGGVSSNDFSSL

Fetching sequences: 100%|██████████| 469/469 [03:35<00:00,  2.18it/s]

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'P05204', 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Non-histone chromosomal protein HMG-17'}}, 'alternativeNames': [{'fullName': {'value': 'High mobility group nucleosome-binding domain-containing protein 2'}}]}, 'genes': [{'geneName': {'value': 'HMGN2'}, 'synonyms': [{'value': 'HMG17'}]}], 'sequence': {'value': 'MPKRKAEGDAKGDKAKVKDEPQRRSARLSAKPAPPKPEPKPKKAPAKKGEKVPKGKKGKADAGKEGNNPAENGDAKTDQAQKAEGAGDAK', 'length': 90, 'molWeight': 9393, 'crc64': 'B1C58E52200422C8', 'md5': 'A9CFC49F70FC3ED141571BAB89415E5B'}, 'extraAttributes': {'uniParcId': 'UPI000013EBD7'}}

Results summary:
- Total proteins processed: 469
- Successfully retrieved: 466 sequences

Output files:
- paper2_fasta_results.csv: Contains all retrieved entries with gene names, accessions, descriptions, and sequences
- paper2_fasta_results.fasta: Contains protein sequences in FASTA format with gene names in headers
